In [1]:
import torch
from utils import get_embedding
import pandas as pd
from tqdm import tqdm
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import csv

In [2]:
# embedding = get_embedding()

In [3]:
# fn = "thucnews/thucnews.csv"
# df = pd.read_csv(fn)
# df.shape

In [4]:
# title_embeddings = torch.zeros(df.shape[0], 1024)
# title_embeddings.shape

In [5]:
# titles = df['title']
# with torch.no_grad():
#     i = 0
#     for title in tqdm(titles):
#         t_em = torch.tensor(embedding.embed_query(title), dtype=float)
#         title_embeddings[i, :] = t_em
#         i = i + 1
#     print(i)

In [6]:
# with open("thucnews/t2v_large/pkl/title_embeddings_large.pickle", "wb") as file:
#     pickle.dump(obj=title_embeddings, file=file)

In [7]:
with open("thucnews/t2v_large/pkl/title_embeddings_large.pickle", "rb") as file:
    title_embeddings = pickle.load(file)

In [8]:
title_embeddings.shape

torch.Size([3163, 1024])

In [9]:
score = cosine_similarity(title_embeddings, title_embeddings)

In [15]:
threshold = 0.7
percentage = np.sum(score > threshold ) / (score.size)
print("percentage: ", percentage)
print("np.sum(score >= threshold):", np.sum(score >= threshold))
score[score < threshold] = 0
np.fill_diagonal(score, 1.0)

percentage:  0.00036213454072834124
np.sum(score >= threshold): 3623


In [16]:
save_path = f"thucnews/t2v_large/thucnews_title_{threshold}.csv"
# 打开CSV文件
with open(save_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # 写入表头
    writer.writerow(['Row', 'Column', 'Value'])

    # 遍历数组并写入CSV文件
    for i in tqdm(range(score.shape[0])):
        for j in range(score.shape[1]):
            if score[i][j] > 0:
                writer.writerow([i, j, score[i, j]])

100%|██████████| 3163/3163 [00:10<00:00, 299.95it/s]


In [17]:
df = pd.read_csv(save_path)
df.shape

(3623, 3)

In [18]:
df.columns

Index(['Row', 'Column', 'Value'], dtype='object')

In [19]:
df.to_csv(save_path, sep=" ", header=False, index=False)